<a href="https://colab.research.google.com/github/Sangyups/PyTorchZeroToAll/blob/main/lec_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
drive.mount('/content/gdrive')


class DiabetesDataset(Dataset):
    def __init__(self):
        xy = np.loadtxt('/content/gdrive/My Drive/Colab Notebooks/diabetes.csv', delimiter=',', dtype=np.float32)
        self.x_data = Variable(torch.from_numpy(xy[:, 0:-1]))
        self.y_data = Variable(torch.from_numpy(xy[:, [-1]]))
        self.len = xy.shape[0]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        return self.len

dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred
model = Model()

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        y_pred = model(inputs)

        loss = criterion(y_pred, labels)
        print(epoch, i, loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
0 0 0.7404325604438782
0 1 0.769923210144043
0 2 0.7774761915206909
0 3 0.7856287360191345
0 4 0.7279067635536194
0 5 0.7103900909423828
0 6 0.7306411862373352
0 7 0.6982062458992004
0 8 0.7002115249633789
0 9 0.6969097852706909
0 10 0.691781759262085
0 11 0.6899008750915527
0 12 0.6827855706214905
0 13 0.6887023448944092
0 14 0.6750074625015259
0 15 0.6947441101074219
0 16 0.675614595413208
0 17 0.6534983515739441
0 18 0.6608818173408508
0 19 0.6711641550064087
0 20 0.6853827834129333
0 21 0.6398370862007141
0 22 0.6758054494857788
0 23 0.647378146648407
1 0 0.647056519985199
1 1 0.6655683517456055
1 2 0.7073807120323181
1 3 0.6358664631843567
1 4 0.6316022872924805
1 5 0.6518876552581787
1 6 0.7234415411949158
1 7 0.641357958316803
1 8 0.6158144474029541
1 9 0.6626518964767456
1 10 0.6626575589179993
1 11 0.5828748941421509
1 12 0.6761072278022766
1 13 0.